In [1]:
import vcf
import io
import pandas as pd
import pysam
from cyvcf2 import VCF
from gtfparse import read_gtf
from tqdm import tqdm
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
trans=[]#create list of PID with rna-seq data in heidelberg
with open('trans.txt')as s:
    v=s.readlines()[1:]
    for rec in v:
        rec=rec.rstrip()
        trans.append(rec.split(' ')[-1].split('-')[1])


df=pd.read_csv('2020-01-28-AR-IntronExonJunctions_387_KGE_Genes.variant_info.vcf',skiprows=108,sep='\t')
df_genes=pd.read_excel('2021-01-28_cancer_gene_list_germline_splice.xlsx')



df['PID_trans']=df['PID'].str.split('-',expand=True)[1]
df=df[df['PID_trans'].isin(trans)]


#naga=set(df['HUGO_Symbol']) ##gene symbols which cann ot be matched between naga and arne, no hits or renaming ?
#for g,temp in zip(df_genes['Approved symbol (HGNC)'],df_genes['Splice Project Gene Priority']):
 #   if g not in naga:
  #      print(g,temp)
#naga=set(df['HUGO_Symbol']) #gene symbols which cann ot be matched between naga and arne, no hits or renaming ?
#for g in df_genes['Approved symbol (HGNC)']:
 #   if g not in naga:
  #      print(g)

df_merge=pd.merge(df,df_genes,left_on='HUGO_Symbol',right_on='Approved symbol (HGNC)', how='left')
df_merge.dropna(subset=['Splice Project Gene Priority'],inplace=True)



hgnc=pd.read_csv('HGNC_trans_symbolLookUp.txt', sep='\t')


#for x,y in hgnc['Approved symbol'].value_counts().items(): # genes with no clear hgnc in my database, dropdupli, keep first
#    if y >1:
 #       print(x)

hgnc=hgnc.drop_duplicates(subset=['Approved symbol'])

replaceDict={'NM_003220':'NM_001042425','NM_004241':'NM_032776','NM_022976':'NM_001144914'}
for _ in replaceDict:
    i=hgnc[hgnc['RefSeq accession']==_].index.values[0]
    hgnc.at[i,'RefSeq accession']=replaceDict[_]
#discrepancy between https://biomart.genenames.org/ and https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg19
# need to update the refseq trans


df_merge=pd.merge(df_merge,hgnc,left_on='HUGO_Symbol', right_on='Approved symbol', how='inner')



#set(df_merge[~df_merge['RefSeq accession'].isin(set(gtf_df['transcript_id']))]['RefSeq accession']) genes were hgnc trans can not be matched between gtf and df_merge

gtf_df=read_gtf('/mnt/g27prist/CMTD/Stephan/bcbio_installation/genomes/Hsapiens/GRCh37/StephanDatabase/GRCH37/annotation/GRCh37_latest_genomic_replace.gtf')
gtf_df['transcript_id']=gtf_df['transcript_id'].str.split('.',expand=True)[0]
gtf_df=gtf_df[gtf_df['transcript_id'].isin(set(df_merge['RefSeq accession']))]

df_merge=df_merge.drop_duplicates(subset=['#CHROM','POS','ALT','PID'])

/software/miniconda3/envs/Jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:Extracted GTF attributes: ['gene_id', 'db_xref', 'description', 'gbkey', 'gene', 'gene_biotype', 'pseudo', 'transcript_id', 'product', 'exon_number', 'gene_synonym', 'tag', 'protein_id', 'exception', 'inference', 'note', 'partial', 'The', 'transl_except', 'isoform', 'anticodon', 'non-AUG', 'standard_name', 'source', 'similar', 'transferase', 'codons', '12S', '16S', 'transl_table', 'ATPase']


In [3]:
def exonPos(trans,pos):
    temp=gtf_df[(gtf_df['transcript_id']==trans)&(gtf_df['seqname'].str.len()<=3)&(gtf_df['feature']=='CDS')]
    if temp.empty:
        print(trans)
        return('err','err')
    strand=temp['strand'].values[0]

    if strand=='+':
        StartEx=temp['start'].min()
        StopEx=temp['end'].max()
    if strand=='-':
        StartEx=temp['end'].max()
        StopEx=temp['start'].min()
    
    
    if strand == '+':
        if pos < StartEx and pos > (StartEx -30):
            dist=StartEx-pos
            return (dist, '5utr_startcodon')
        if pos > StopEx and pos < (StopEx + 30):
            dist=pos-StopEx
            return (dist, '3utr_stopcodon')
        
        OutsideAcceptor=temp[((temp['start']-30)<=pos)&((temp['start'])>pos)]
        InsideAcceptor=temp[((temp['start']+30)>=pos)&((temp['start'])<=pos)]
        
        OutsideDonor=temp[((temp['end']+30)>=pos)&((temp['end'])<pos)]
        InsideDonor=temp[((temp['end']-30)<=pos)&((temp['end'])>=pos)]
        
        if not OutsideAcceptor.empty:
            dist=OutsideAcceptor['start'].values[0]-pos
            return (dist,'outsideAcceptorSite')
        if not InsideAcceptor.empty:
            dist=pos-InsideAcceptor['start'].values[0]
            return (dist,'insideAcceptorSite')
            
        
        if not OutsideDonor.empty:
            dist=pos-OutsideDonor['end'].values[0]
            return (dist,'outsideDonorSite')
            
        if not InsideDonor.empty:
            dist=InsideDonor['end'].values[0]-pos
            return (dist,'insideDonorSite')
    
    
    if strand =='-':
        if pos > StartEx and pos < (StartEx + 30):
            dist=pos-StartEx
            return (dist, '5utr_startcodon')
        if pos < StopEx and pos > (StopEx+30):
            dist=StopEx-pos
            return (dist, '3utr_stopcodon')
        
        OutsideDonor=temp[((temp['start']-30)<=pos)&((temp['start'])>pos)]
        InsideDonor=temp[((temp['start']+30)>=pos)&((temp['start'])<=pos)]
        
        OutsideAcceptor=temp[((temp['end']+30)>=pos)&((temp['end'])<pos)]
        InsideAcceptor=temp[((temp['end']-30)<=pos)&((temp['end'])>=pos)]
        
        if not OutsideAcceptor.empty:
            dist=pos-OutsideAcceptor['end'].values[0]
            return (dist,'outsideAcceptorSite')
        if not InsideAcceptor.empty:
            dist=InsideAcceptor['end'].values[0]-pos
            return (dist,'insideAcceptorSite')
            
        
        if not OutsideDonor.empty:
            dist=OutsideDonor['start'].values[0]-pos
            return (dist,'outsideDonor')
            
        if not InsideDonor.empty:
            dist=pos-InsideDonor['start'].values[0]
            return (dist,'insideDonor')
    
    return ('01','01')
            
    
    
    
        
    
posRelativeRefSeq = []
Utr_cod=[]

for pos,trans in tqdm(zip(df_merge["POS"],df_merge["RefSeq accession"]),total=len(df_merge)):
    _=exonPos(trans,int(pos))
    posRelativeRefSeq.append(_[0])
    Utr_cod.append(_[1])
#df_merge['PosExonRefSeqAccession']=posRelativeRefSeq

df_merge['PosExonRefSeqAccession']=posRelativeRefSeq
df_merge['PosExon_type']=Utr_cod

df_merge=df_merge[df_merge['PosExon_type']!='01']

100%|██████████| 19062/19062 [02:07<00:00, 149.53it/s]


In [4]:
df_merge.to_pickle('dfmerge')

In [5]:
df_merge

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Control_GT,Tumor_GT,HUGO_Symbol,Variant_Classification,HGVSg,HGVSc,HGVSp,Allele_Frequency,VEP_Most_Severe_Consequence,Positive_Evidence,Negative_Evidence,Positive_CharGer_Score,Negative_CharGer_Score,CharGer_Score,ClinVar_Pathogenicity,ACMG_Classification,CharGer_Classification,PubMed_Link,ClinVar_Traits,CharGer_Summary,Control_VAF,Tumor_VAF,Tumor_dpALT,Tumor_dp,Control_dpALT,Control_dp,GT_Classification,max_gnomAD_AF,exome_control_AF,local_control_AF,PID,overlap,canopyCluster,TiN_Class,CHR_ARM_INFO,AI_cluster,thresholdCluster,GIT_VERSION,ALL_FUNCTIONAL_CONSENSUS,RNA_EXPRESSION,RNA_INFO,RNA_VAF,PID_trans,Approved symbol (HGNC),Splice Project Gene Priority,"Origin (Bertelsen 2019 (PMID: 31263571), Schrader 2016 (PMC5477989), Huang 2018 (PMID: 29625052), Groebner 2018 (PMID: 29489754), Priestley 2019 (PMID: 31645765))",NCBI gene ID (HGNC),RefSeq (HGNC),Total Exon Count (hg19),Coding Exon Count (hg19),Protein size [aa] (hg19),Gene Basket (NCT),Gene classification (most likely mechanism),Gene group (KGE),inheritance cancer predisposition,OMIM-G Phenotypes,non-cancer related phenotype,final proposal,technical details gene,Huang_KGE_Cancer syndrome(s),Huang_KGE_Major associated tumor types,Literature,HGNC ID,Status,Approved symbol,Approved name,RefSeq accession,PosExonRefSeqAccession,PosExon_type
0,2,169853169,.,T,C,2584,PASS,BRF=0.26;FR=0.5;HP=1;HapScore=2;MGOF=1;MMLQ=38...,GT:GL:GOF:GQ:NR:NV,"1/0:-133.02,0,-122.02:1:99:102:55","1/0:-130.1,0,-172.6:0:99:115:52",ABCB11,synonymous_variant,2:g.169853169T>C,ENST00000263817:c.453T>C,ENSP00000263817:p.151,0.000009,synonymous_variant,PM2,.,2,0,2,.,Uncertain Significance,Uncertain Significance,.,.,PM2(Low/absent allele frequency 9.479e-06<=0.0...,0.452174,0.539216,55,102,52,115,GermlineInBoth,1.0025e-05,.,.,D021-592D9X,1,1,Germline_AI_neutral,2q,1,Germline,v0.2.2,NaN,NaN,NaN,NaN,592D9X,ABCB11,Cold (387-ACMG-MASTER),"P, G, H",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cholestasis, progressive familial intrahepatic...",NaN,NaN,NaN,NaN,NaN,NaN,HGNC:42,Approved,ABCB11,ATP binding cassette subfamily B member 11,NM_003742,24,insideDonor
1,2,169826057,.,T,G,2625,PASS,BRF=0.1;FR=0.5;HP=1;HapScore=2;MGOF=1;MMLQ=37;...,GT:GL:GOF:GQ:NR:NV,"1/0:-118.32,0,-133.02:1:99:78:37","1/0:-148.9,0,-170.7:0:99:103:49",ABCB11,missense_variant,2:g.169826057T>G,ENST00000263817:c.1814T>G,ENSP00000263817:p.Q605P,0.000000,missense_variant,"PM2,PP2,PP3",.,4,0,4,.,Uncertain Significance,Uncertain Significance,.,.,PM2(Low/absent allele frequency 0.0<=0.0005) -...,0.474359,0.475728,49,103,37,78,GermlineInBoth,.,.,.,H021-06H1,1,1,Germline_AI_neutral,2q,1,Germline,v0.2.2,NaN,NaN,NaN,NaN,06H1,ABCB11,Cold (387-ACMG-MASTER),"P, G, H",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cholestasis, progressive familial intrahepatic...",NaN,NaN,NaN,NaN,NaN,NaN,HGNC:42,Approved,ABCB11,ATP binding cassette subfamily B member 11,NM_003742,4,insideAcceptorSite
2,2,169870004,.,G,A,2157,PASS,BRF=0.18;FR=0.5;HP=2;HapScore=2;MGOF=5;MMLQ=37...,GT:GL:GOF:GQ:NR:NV,"1/0:-110.9,0,-139.5:5:99:79:34","1/0:-109.48,0,-138.28:5:99:80:36",ABCB11,missense_variant,2:g.169870004G>A,ENST00000263817:c.167G>A,ENSP00000263817:p.S56L,0.000555,missense_variant,"PP2,PP3",BP4,2,-1,1,Uncertain Significance,Uncertain Significance,Uncertain Significance,.,not specified;not provided,PP2(Missense variant in gene from PP2 gene lis...,0.430380,0.450000,36,80,34,79,GermlineInBoth,0.00054664,.,.,H021-13JVXT,1,1,Germline_AI_neutral,2q,1,Germline,v0.2.2,NaN,NaN,NaN,NaN,13JVXT,ABCB11,Cold (387-ACMG-MASTER),"P, G, H",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cholestasis, progressive familial intrahepatic...",NaN,NaN,NaN,NaN,NaN,NaN,HGNC:42,Approved,ABCB11,ATP binding cassette subfamily B member 11,NM_003742,16,insideAcceptorSite
3,2,169791916,.,C,T,2965,PASS,BRF=0.11;FR=0.5;HP=1;HapScore=2;MGOF=7;MMLQ=36...,GT:GL:GOF:GQ:NR:NV,"0/1:-252.04,0,-216.74:7:99:180:95","0/1:-247.73,0,-292.93:4:99:209:97",ABCB11,missense_variant,2:g.169791916C>T,ENST000002638